In [2]:
import os
import os.path as op
import glob
import numpy as np
import multiprocessing
from nipype.interfaces import fsl

In [3]:
#preliminary housekeeping
home_dir = '/data/home/iballard/fd/'
subj_file = home_dir + 'subjects.txt'
subs = list(np.loadtxt(subj_file,'string'))
os.chdir(home_dir)
exps = ['sim_4mm']
runs = map(str,range(1,4))

In [7]:
#get cortical areas from Yeo atlas
path = home_dir + '/data/fsaverage/label/'

#get cortical areas from Michael's parcellation
cortical_areas =  glob.glob(path + '*yeo*')
cortical_areas = [x.split('/')[-1] for x in cortical_areas]

In [50]:
def make_mask(in_tuple):
    sub,roi = in_tuple
    roi_id = roi.split('.')[1]
    
    cmd = ['make_masks.py',
           '-s',
           sub,
           '-roi',
           roi_id,
            '-label',
           roi_id,
           '-sample graymid',
           '-exp',exps[0],
           '-serial']
    cmd = ' '.join(cmd)
    os.system(cmd)
    print cmd

In [54]:
in_tuples = []
for sub in subs:
    for roi in cortical_areas:
        in_tuples.append((sub,roi))

In [55]:
pool = multiprocessing.Pool(32)
pool.map(make_mask,in_tuples)
pool.terminate()
pool.join()